In [338]:
# IMPORTAZIONE LIBRERIE NECESSARIE ALL'ANALISI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# IMPOSTAZIONE STILI E VISUALIZZAZIONE
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

# Importazione Dataset da CSV, Analisi iniziale e Pulizia dataframe

In [339]:
# CARICO I DATI NEL DATAFRAME df
df=pd.read_csv('winemag-data-130k-v2.csv') 

In [ ]:
df.head()      

In [ ]:
df.info()

DALLE PRIME INFO SI EVINCE CHE:
- ABBIAMO 3 DATI DI TIPO NUMERICO, QUELLI RELATIVO AI PUNTI (INT), QUELLI RELATIVI AL PREZZO (FLOAT) E UNA COLONNA UNNAMED: 0 CHE SEMBREREBBE ESSERE LA DUPLICAZIONE DELL'INDICE. GLI ALTRI DATI SONO DI TIPO OGGETTO.
- IN GENERALE NON CI SONO DATI PER CUI ANDREBBE CAMBIATA LA TIPOLOGIA
- CI SONO ALCUNI VALORI NULLI CHE ANDIAMO AD ANALIZZARE PIù IN DETTAGLIO

In [ ]:
# VADO A CONTROLLARE I VALORI NULLI
df.isnull().sum().sort_values(ascending=False)

SICCOME L'OBBIETTIVO FINALE E' QUELLO DI COSTRUIRE UN MARKETPLACE E SICCOME LA MAGGIOR PARTE DELLE ANALISI SARANNO BASATE SUI PREZZI
- DECIDO DI SCARTARE TUTTE LE RIGHE DI VALORI IN CUI NON ABBIAMO IL PREZZO
- VADO A DROPPARE ALCUNE COLONNE CHE NON RITENGO RILEVANTI AL MIO CASO, OVVERO: Unnamed: 0, region_2 (CHE HA PIù DELLA METà DEI VALORI NULLI), taster_twitter_handle, taster_name 

In [ ]:
# SCARTO LE RIGHE IN CUI PRICE è NULLO
df=df[~df['price'].isna()]
df.shape
# SIAMO SCESI A 120975 RIGHE

In [344]:
# DROPPO LE COLONNE NON RITENUTE UTILI ALL'ANALISI
df=df.drop(['Unnamed: 0','region_2','taster_name','taster_twitter_handle'], axis=1)    

In [ ]:
# CONTROLLO CHE NON CI SIANO DUPLICATI
df[df.duplicated()].shape

# VEDIAMO CHE CI SONO 9382 RIGHE DUPLICATE CHE ANDRò A RIMUOVERE DAL DATAFRAME PER AVERE UNA ANALISI PIù ACCURATA

In [346]:
# RIMUOVO I DUPLICATI
df=df.loc[~df.duplicated()]

In [ ]:
df.shape
# ORA ABBIAMO 111593 RIGHE DI DATI

# Esplorazione feature

CERCO DI OTTENERE UNA DESCRIZIONE PIù APPROFONDITA DI TUTTI I DATI.  

IN PARTICOLARE MI INTERESSA INDAGARE SULLE CARATTERISTICHE 'PRICE' E 'POINTS' PER VEDERE SE I DATI SONO DISTRIBUITI EQUAMENTE E SE CI SONO DEGLI OUTLIER

In [ ]:

df.describe(include='all')

DAL METODO DESCRIBE RIESCO A TIRAR FUORI INFORMAZIONI PREZIOSE, IN PARTICOLARE I RISULTATI MI DICONO CHE:
- CI SONO 42 DIVERSI PAESI PRESI IN ESAME:
       IL PIU' POPOLARE è US CON OLTRE 50MILA RECENSIONI (QUASI LA METà)
- CI SONO CIRCA 700 VARIETà DI VINO DIVERSE:
       LA PIù POPOLARE è IL PINOT NOIR CON QUASI 12MILA DATI (PIù DI UN DECIMO DEL TOTALE)
- CI SONO QUASI 16 MILA DIVERSE VIGNE:
       LA PIù POPOLARE (WINES & WINEMAKERS) HA 204 RECENSIONI
- I PUNTI VANNO DA 80 A 100 CON UNA MEDIA DI 88:
       LA DEVIAZIONE STANDARD ABBASTANZA BASSA (3) CI SUGGERISCE CHE QUESTO DATO è BEN DISTRIBUITO
- I PREZZI VANNO DA 4 A 3300 USD:
       LA MEDIA è DI CIRCA 36 USD: 
          CON IL 75% DEI DATI SOTTO I 42 USD E UNA DEVIAZIONE STANDARD MOLTO ALTA (42) QUESTO CI FA INTUIRE CHE CI SONO DEGLI OUTLIER DI PREZZO


In [ ]:
# APPROFONDISCO LA FEATURE 'POINTS' GRAFICANDO LA DISTRIBUZIONE DEI PUNTEGGI PER VEDERE QUALI SONO I PIù COMUNI
plt.figure(figsize=(10, 6))
sns.histplot(df['points'], bins=20, kde=True, color='skyblue')
plt.title('Distribuzione dei punteggi (Points)')
plt.xlabel('Punteggio')
plt.ylabel('Frequenza')
plt.show()

DAL GRAFICO E DALLE STATISTICHE DEI PUNTEGGI SI EVINCE UNA DISTRIBUZIONE GAUSSIANA, IN CUI LA MAGGIOR PARTE DEI DATI HA VALORE TRA 86 E 91 PUNTI

In [ ]:
# APPROFONDISCO LA FEATURE 'PRICE' GRAFICANDO LA DISTRIBUZIONE DEI PREZZI PER VEDERE QUALI SONO I PIù COMUNI
Q99=df['price'].quantile(0.99)
df_filtered_price=df[df['price'] < Q99] # per una visualizzazione più chiara escludo gli outlier maggiori filtrando i vini che hanno prezzo inferiore al 99% dei dati

plt.figure(figsize=(10, 6))
sns.histplot(df_filtered_price['price'], bins=30, kde=True, color='salmon')
plt.title('Distribuzione dei prezzi (Price)')
plt.xlabel('Prezzo')
plt.ylabel('Frequenza')
plt.show()

DAL GRAFICO E DALLE STATISTICHE DEI PREZZI VEDIAMO COME LA MAGGIOR PARTE DEI DATI HA PREZZI COMPRESI TRA I 15 E I 40 USD, CHE SOLO L'1% DEI DATI HA PREZZO SUPERIORE A 160 USD (IN PARTICOLARE LA 'CODA' DEL GRAFICO INIZIA GIà INTORNO A 80 USD), RAFFORZANDO L'IDEA DEI POCHI OUTLIER DI PREZZO MA MOLTO ESTREMI.

In [ ]:

df_unfiltered_price=df[df['price'] > Q99]

plt.figure(figsize=(10,6))
sns.histplot(df_unfiltered_price['price'], bins=30, kde=True)

In [ ]:
# INDIVIDUO MEDIA PUNTI E LA MEDIANA DEI PREZZI (COSI' DA NON TENERE CONTO DEGLI OUTLIER DI PREZZO) 
# PER CAPIRE VELOCEMENTE SE UN VINO HA UN BUON PREZZO ED E' DI BUONA QUALITA'
points_mean=df['points'].mean()     
price_median=df['price'].median()
print(f'{points_mean}, {price_median}')

In [353]:
Q1=df['price'].quantile(0.25)
Q3=df['price'].quantile(0.75)


In [354]:
# STABILISCO DELLE FASCE DI PREZZO
df['price_range']=''
df.loc[df['price']<= Q1,['price_range']]='cheap'     # FASCIA ECONOMICA--> NEL PRIMO QUARTILE
df.loc[(df['price']>Q1) & (df['price']<=Q3),['price_range']]='medium'       # FASCIA MEDIA--> SECONDO E TERZO QUARTILE
df.loc[(df['price']>Q3) & (df['price']<=Q99),['price_range']]='high'      # FASCIA ALTA--> NELL'ULTIMO QUARTILE
df.loc[df['price']>Q99,['price_range']]='expensive'     # FASCIA DEGLI OUTLIER

SICCOME IL MARKETPLACE VUOLE MIRARE AD OFFRIRE VINI DI QUALITà AL MIGLIOR PREZZO, VADO A SCARTARE DAL DATAFRAME TUTTI I DATI CHE HO CONSTATATO ESSERE DEGLI OUTLIER ESTREMI

In [355]:
df_expensive=df[df['price_range']=='expensive']

In [356]:
df=df[df['price_range']!='expensive']

ANALIZZO LA DISTRIBUZIONE DEI VINI IN BASE AL PAESE DI PROVENIENZA 'COUNTRY', 
SICCOME HO TANTI DATI, PER RENDERE IL GRAFICO PIù LEGGIBILE PRENDO COME RIFERIMENTO I PRIMI 10 PAESI PER FREQUENZA

In [ ]:
country_counts = df['country'].value_counts().head(10)  

plt.figure(figsize=(12, 6))
sns.barplot(x=country_counts.values, y=country_counts.index, palette='viridis')
plt.title('Top 10 Paesi per Numero di Vini')
plt.xlabel('Numero di Vini')
plt.ylabel('Paese')
plt.show()

DAL GRAFICO VEDIAMO CHE US è IL COUNTRY PIù RAPPRESENTATO SEGUITO DA FRANCIA E ITALIA. 
  QUESTE 3 NAZIONI HANNO UNA FORTE INFLUENZA SUI RISULTATI DELLE STATISTICHE.

MA QUALI SONO QUELLI CON I PUNTEGGI MIGLIORI?


In [285]:
# INDIVIDUO I PAESI PIU' POPOLARI, QUELLI CHE HANNO ALMENO L'1% DI RECENSIONI TOTALI (CIRCA 1000) E LI SALVO IN UNA LISTA
popular_countries=df['country'].value_counts().sort_values(ascending=False)
popular_countries=popular_countries[popular_countries>1000]

popular_countries_list=popular_countries.index.to_list()

In [ ]:
# COSTRUISCO UN DF DEI SOLI PAESI PIù POPOLARI
popular_countries_df=df[df['country'].isin(popular_countries_list)]

# GRAFICO LA MEDIA PUNTI DEI PAESI PIù POPOLARI
country_avg_points=popular_countries_df.groupby('country')['points'].agg(['count','mean']).sort_values(by='mean',ascending=False)

plt.figure(figsize=(8,6))
fig=px.scatter(
    country_avg_points,
    x=country_avg_points.index,
    y='mean',
    size='count',
    color=country_avg_points.index,
    size_max=50,
    title='Media punteggio vini dei Paesi più popolari',
    labels={'country':'Paese',
            'mean':'Media punteggio',
            'count':'Numero recensioni'},
)
fig.update_layout(title_x=0.5)  
fig.show()

In [ ]:
popular_countries_df.groupby('country')['points'].describe().sort_values(by='mean',ascending=False)

Dal grafico si vede come la media punteggi va in generale da 86 a 91, i paesi con più recensioni hanno punteggi medi simili (88-89).
Però i paesi con punteggi medi migliori sono Austria e Germania (90)

ANALIZZO LA DISTRIBUZIONE DEI VINI IN BASE ALLA 'variety', 
ANCHE IN QUESTO CASO, PER RENDERE IL GRAFICO PIù LEGGIBILE PRENDO COME RIFERIMENTO LE PRIME 10 VARIETà PER FREQUENZA

In [ ]:
df_variety_counts = df['variety'].value_counts().head(10)

fig = px.bar(
    df_variety_counts,
    x=df_variety_counts.values,
    y=df_variety_counts.index,
    orientation='h',
    title='Top 10 Varietà di Vino',
    labels={'x': 'Numero di Vini', 'y': 'Varietà'},
    color=df_variety_counts.values,
    color_continuous_scale='Blues'
)
fig.show()

ABBIAMO UNA PREVALENZA DI RECENSIONI SU VINI PINOT NOIR, SEGUITO DA CHARDONNAY, CABERNET SAUVIGNON E RED BLEND
SICCOME, COME VISTO IN PRECEDENZA CI SONO PIù DI 700 VARIETà DIFFERENTI ANDRò A CONSIDERARE SOLO QUELLE PIù POPOLARI CHE DOVREBBERO CORRISPONDERE A PRODOTTI A PIù ALTA ROTAZIONE

In [289]:
# INDIVIDUO LE VARIETA' DI VINI PIU' POPOLARI E LI SALVO IN UNA LISTA, VADO A CONSIDERARE SOLO QUELLE VARIETà PER LA QUALI ABBIAMO UN MINIMO DI 1000 DATI (circa 1% del totale)
popular_varieties=df['variety'].value_counts().sort_values(ascending=False)
popular_varieties=popular_varieties[popular_varieties>1000]

popular_varieties_list=popular_varieties.index.to_list()


Costruisco un dataframe con sole le varietà più popolari e vado a fare una classifica in base a quelle che hanno i punteggi medi migliori

In [290]:
popular_varieties_df=df[df['variety'].isin(popular_varieties_list)]

pop_var_avg_points=popular_varieties_df.groupby('variety')['points'].agg(['count','mean']).sort_values(by='mean', ascending=False)

In [ ]:
plt.figure(figsize=(12,8))
fig=px.scatter(
    pop_var_avg_points,
    x=pop_var_avg_points.index,
    y='mean',
    size='count',
    color=pop_var_avg_points.index,
    size_max=30,
    title='Media punteggio vini dei Paesi più popolari',
    labels={'variety':'Varietà Vino',
            'mean':'Media punteggio',
            'count':'Numero recensioni'},
)
fig.update_layout(title_x=0.5)  
fig.show()

Qui vediamo come le qualità più popolari (Pinot noir, Chardonnay, Cabernet-Sauvignon) non sono quelle con i punteggi medi migliori, bensì vini più ricercati come Nebbiolo e Gruner Veltiner con punteggio medio superiore ai 90.

# Relazione tra le Features

CERCO SE C'E' CORRELAZIONE TRA PREZZO E PUNTEGGIO

In [ ]:
#  CORRELAZIONE PREZZO-PUNTEGGIO (GENERALE)
(df.groupby('points')['price'].mean()
 .plot(kind='line',
       ylabel='average price',
       xlabel='points',
       title='Average price by points')
)
plt.show()

In linea generale si vede un progressivo aumento del prezzo medio all'aumentare del punteggio

In [ ]:
#  CORRELAZIONE PREZZO-PUNTEGGIO (PIU' SPECIFICA)
fig=px.scatter(
    df,
    x='points',
    y='price',
    hover_name='designation',
    hover_data=['designation','country','region_1','price','points'],
    color='variety'
)
fig.update_layout(
    title="Price distribution by points",
    title_x=0.5,  
    yaxis_title="Price, USD",
    xaxis_title="Points",
    font=dict(size=12),  
    template="plotly_dark",  
)
fig.show()


Nello specifico, si conferma la tendenza che all'aumentare del prezzo aumenta il punteggio, però si vede che ci sono panche molti vini con punteggio alto e prezzo contenuto, quindi con buon rapporto qualità-prezzo

In [ ]:
# CORRELAZIONE PREZZO-PUNTEGGIO TRAMITE MATRICE DI CORRELAZIONE
corr_mat=df[['points','price']].corr()
corr_mat

In [ ]:
# GRAFICO DELLA CORRELAZIONE CON HEATMAP
plt.figure(figsize=(5,4))
sns.heatmap(corr_mat, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()


VALUTANDO TUTTI I RISULTATI OTTENUTI SI PUO' DIRE CHE C'E' UNA CORRELAZIONE TRA PREZZO E PUNTEGGIO MA NON FORTISSIMA.  
IN PARTICOLARE DALLA MATRICE DI CORRELAZIONE OTTENIAMO 0.55 CHE E' UN VALORE ABBASTANZA INTERMEDIO CHE NON CI PERMETTE DI STABILIRE UNA FORTE PROPORZIONALITà DIRETTA

INIZIO A FILTRARE IL DATAFRAME IN BASE A CIò CHE VOGLIO METTERE NEL MARKETPLACE

IL MIO MARKETPLACE HA COME TARGET CONSUMATORI CHE SONO MOLTO ATTENTI ALLA QUALITà DEI VINI, PERCIò I DATAFRAME FILTRATI SONO SEMPRE ORDINATI PER PUNTEGGIO MAGGIORE, CON UN OCCHIO ANCHE AL PREZZO. SI TRATTA DI PERSONE DISPOSTE A PAGARE PER AVERE UN BUON PRODOTTO, MA NON A STRAPAGARE SE CI SONO ALTERNATIVE CON ALTRETTANTA BUONA QUALITà AD UN PREZZO PIù BASSO.

COSA VOGLIO METTERE NEL MARKETPLACE?

- Varietà di vino piu popolari, perchè dovrebbero corrispondere ai prodotti più richiesti e quindi a più alta rotazione:
    - 10 per ognuna delle 20 varietà più popolari con ottimo rapporto qualità-prezzo
        (prezzo sotto la mediana e punteggio sopra la media)
    - 10 per varietà economici (sotto i 10 USD), ma con punteggio buono
    - 5 per varietà con punteggio più alto
    

In [ ]:
# Identificare vini con punteggi molto alti rispetto alla loro fascia di prezzo
high_score_good_price=df[(df['points'] >= 95) & (df['price']<=price_median)].sort_values(by='points', ascending=False)
high_score_good_price
# SICCOME SONO SOLO 16 E SONO ALTAMENTE RILEVANTI, LI CONSIDERERò TUTTI NEL DATAFRAME FINALE

In [ ]:
# ORDINO IL DATAFRAME DELLE VARIETà PIù POPOLARI PER PUNTI DECRESCENTI
popular_varieties_df=popular_varieties_df.sort_values(by='points',ascending=False)

A SECONDA DEL PREZZO VADO A SUDDIVIDERE ULTERIORMENTE IL DATAFRAME:  

- PRENDO I PRIMI 10 PER PUNTEGGIO NELLA FASCIA 'CHEAP' E 'MEDIUM'
- PRENDO I PRIMI 5 INDIPENDENTEMENTE DAL PREZZO, esclusa la fascia expensive contenente gli outlier

In [297]:
pop_var_cheap=popular_varieties_df[popular_varieties_df['price_range']=='cheap'].groupby('variety').head(10).sort_values(by='variety')
pop_var_medium=popular_varieties_df[popular_varieties_df['price_range']=='medium'].groupby('variety').head(10).sort_values(by='variety')
pop_var_top5=popular_varieties_df.groupby('variety').head(5).sort_values(by='variety')
# i 3 df così ottenuti andranno tutti nel df finale

In [ ]:
# pop_var_cheap
# pop_var_medium
pop_var_top5

PER AVERE UNA BUONA DISTRIBUZIONE PER 'COUNTRY' VADO A PRENDERE I PRIMI 10 VINI PER OGNUNA DELLE NAZIONI PIù POPOLARI

In [299]:
# DF dei TOP 10 delle nazioni più popolari
best_rated_best_countries=(df[df['country'].isin(popular_countries_list)]
 .sort_values(by='points',ascending=False)
 .groupby('country')
 .head(10)
 .sort_values(by='country')
 )
#best_rated_best_countries

# ANCHE QUESTO df ANDREMO AD AGGIUNGERLO AL df finale

In [ ]:
# GRAFICO DEI MIGLIORI 10 VINI DELLE NAZIONI PIU' POPOLARI
fig=px.scatter(
    best_rated_best_countries,
    x='variety',
    y='points',
    color='country',
    width=1200,
    height=600,
    hover_name='designation',
    hover_data={
        'designation':True,
        'country':True,
        'points':True,
        'price':True
    },
    size='points',
    size_max=10
)
fig.update_layout(
    title="Best Rated Wines by Country and Variety",
    title_x=0.5,  
    xaxis_title="Wine Variety",
    yaxis_title="Points",
    font=dict(size=12),  
    template="plotly_dark",  
)
# Migliora la leggibilità delle etichette degli assi
fig.update_xaxes(tickangle=45)  
fig.update_yaxes(tickformat=".1f")  

fig.show()

VOGLIO ORA GARANTIRE UNA BUONA DISTRIBUZIONE DI TIPOLOGIE DI VINI BASATE SU PAROLE CHIAVI CHE COMUNEMENTE SI USANO PER DESCRIVERLI.  
VADO A TROVARE NEL DATAFRAME QUEI VINI PER LA QUALE NELLA 'description' COMPARE LA PARTICOLARE KEYWORD RICERCATA

In [301]:
# IMPOSTO DELLE PAROLE CHIAVI, TRA LE PIU' COMUNI, PER DESCIVERE I VINI
keywords = [
    'fruity','dry','aged','spicy','balanced','tannic','rich','intense'
]

In [ ]:
# Funzione che accetta come parametro una parola chiave (inserita come stringa tra apici) 
# e restituisce i grafici di una classifica:
# - dei top 3 vini contenenti tale parola chiave nelle recensioni
# - dei top 10 vini contenenti tale parola chiave e con buon rapporto qualità-prezzo

def keywords_func(key,price_median, points_mean):
    # Filtro in base alla parola chiave
    key_df = df[df['description'].str.contains(key, case=False, na=False)]

    # Rimuovo i duplicati
    key_df = key_df[~key_df.duplicated(subset='designation')]

    # Duplico il dataframe per separare l'analisi in: top e value for money
    key_df_vfm=key_df

    # Seleziono i top 3 risultati
    key_df = key_df.sort_values(by='points', ascending=False).head(3)

    # Filtro ulteriormente i dati per il vfm
    key_df_vfm = key_df_vfm[(key_df_vfm['price'] < price_median) & (key_df_vfm['points'] > points_mean)]

    # Seleziono i top 10 risultati vfm
    key_df_vfm = key_df_vfm.sort_values(by='points', ascending=False).head(10)

    # Creazione del grafico top
    fig = px.bar(
        key_df,
        x='points',
        y='designation',
        orientation='h',
        color='variety',
        hover_name='designation',
        hover_data={
            'price', 'country', 'winery', 'region_1'
        },
        range_x=[80, 100],
        title=f'Top 3 {key.capitalize()} Wines',
        template="plotly_dark"
    )
    fig.update_layout(
        xaxis_title="Points",
        yaxis_title="Wine",
        yaxis=dict(autorange="reversed"),
        showlegend=True 
    )  
    # Mostra il grafico top
    fig.show()

    # Creazione del grafico vfm
    fig_vfm = px.bar(
        key_df_vfm,
        x='points',
        y='designation',
        orientation='h',
        color='variety',
        hover_name='designation',
        hover_data={
            'price', 'country', 'winery', 'region_1'
        },
        range_x=[80, 100],
        title=f'Top 10 value for money {key.capitalize()} Wines',
        template="plotly_dark"
    )
    fig_vfm.update_layout(
        xaxis_title="Points",
        yaxis_title="Wine",
        yaxis=dict(autorange="reversed"),
        showlegend=True  # Aggiungi legenda
    )
    #Mostra il grafico vfm
    fig_vfm.show()
    return key_df, key_df_vfm   # Return dei due dataframe ottenuti

# Inizializzazione di un dizionario di 'dataframe per keyword', sia top che vfm
keywords_df={}  
keywords_df_vfm={}
dfs_to_concat = []

for i in keywords:      # per ogni parola contenuta nella lista 'keywords:
    key_df,key_df_vfm=keywords_func(i,price_median, points_mean)    # esegue la funzione su ogni parola chiave e li assegna ai dataframe
    # Aggiungo il dataframe dei vini che contengono quella keyword al dizionario di dataframe, sia top che vfm
    keywords_df[i]=key_df    
    keywords_df_vfm[i]=key_df_vfm
    print(f'Best {i} wines: \n')
    print(keywords_df[i][['designation','country','region_1','points','price']])
    print('\n')
    print(f'Best {i} wines value for money: \n')
    print(keywords_df_vfm[i][['designation','country','region_1','points','price']])
    print('\n')
    dfs_to_concat.extend([keywords_df[i], keywords_df_vfm[i]])

In generale si può vedere come le varietà dei vini selezionati per parole chiave cambiamìno a seconda della fascia di prezzo considerata.

VADO AD ANALIZZARE ANCHE LE VIGNE, EVIDENZIANDO QUELLE PER LE QUALI ABBIAMO PIù DATI E QUELLE CHE HANNO PUNTEGGI MIGLIORI

In [303]:
# TROVO LE VIGNE PIU' POPOLARI E LE METTO IN UNA LISTA
popular_wineries=df['winery'].value_counts().sort_values(ascending=False)
popular_wineries=popular_wineries.head(20)
popular_wineries_list=popular_wineries.index.to_list()

In [304]:
# CREO UN DATAFRAME CON I DATI DELLE SOLE VIGNE PIù POPOLARI
popular_wineries_df=df[df['winery'].isin(popular_wineries_list)].sort_values(by='points',ascending=False)

In [ ]:
# E VADO POI A FILTRARLI IN BASE ALLA FASCIA DI PREZZO
pop_wineries_low_cost=popular_wineries_df[popular_wineries_df['price_range']=='cheap'].groupby('winery').head(5).sort_values(by='winery')
pop_wineries_low_cost

In [ ]:
pop_wineries_mid_price=popular_wineries_df[(popular_wineries_df['price_range']=='medium')].groupby('winery').head(5).sort_values(by='winery')
pop_wineries_mid_price

In [ ]:
pop_winery_top=popular_wineries_df.groupby('winery').head(2).sort_values(by='winery')
pop_winery_top

# TUTTI E 3 I df COSì OTTENUTI SARANNO AGGIUNTI AL df FINALE

Vorrei considerare anche le vigne meno popolari con punteggi molto alti e che quindi sfuggirebbero alla classificazione precedente, per andare ad includere dei prodotti più ricercati, per veri intenditori.

In [ ]:
# quali sono le vigne meno conosciute con punteggi ottimi?
less_known_wineries=df[~df['winery'].isin(popular_wineries_list)]
less_known_wineries=(less_known_wineries[less_known_wineries['points']>98]
                     .sort_values(by='points')
                     .groupby('winery').head(2))
less_known_wineries

# ANCHE QUESTO VERRà AGGIUNTO AL df FINALE


COSTRUISCO IL DATAFRAME FINALE, CONCATENANDO TUTTI I RISULTATI FILTRATI DAL DF ORIGINARIO

In [310]:
# COSTRUISCO IL DATAFRAME FINALE
final_df=pd.concat([high_score_good_price,pop_var_cheap,pop_var_medium,pop_var_top5,best_rated_best_countries,pop_wineries_low_cost,pop_wineries_mid_price,pop_winery_top,less_known_wineries], ignore_index=True)

In [311]:
# Concateno tutti i DataFrame in un unico Dataframe Finale
final_df = pd.concat([final_df]+dfs_to_concat, ignore_index=True)

In [ ]:
final_df

ANALISI DATAFRAME FINALE

Siccome la concatenazione potrebbe aver generato dei duplicati andiamo a fare una ulteriore analisi sul dataframe finale

In [ ]:
final_df.shape

In [ ]:
final_df.describe(include='all')

In [ ]:
final_df.info()

In [365]:
final_df=final_df[~final_df.duplicated()]   # ELIMINO I DUPLICATI

In [ ]:
final_df.shape

Nel Dataframe finale abbiamo 887 vini diversi!

In [ ]:
# GRAFICO DI VISIONE GENERALE DEL DATAFRAME FINALE
fig=px.scatter(
    final_df,
    x='points',
    y='price',
    color='variety',
    hover_name='designation',
    hover_data={
        'designation',
        'country',
        'region_1',
        'price',
        'points',
        'price_range',
    }    
)
fig.update_layout(
    title="Wine Marketplace",
    title_x=0.5,  
    xaxis_title="Points",
    yaxis_title="Price",
    font=dict(size=12),  
    template="plotly_dark",  
)
fig.show()

In generale si conferma la tendenza che all'aumentare del punteggio aumenta il prezzo, non mancano comunque prodotti di ottimo rapporto qualità prezzo

In [ ]:
# Top Varietà per Punteggio Medio

top_varieties = final_df.groupby('variety')['points'].mean().sort_values(ascending=False)
plt.figure(figsize=(12, 16))
sns.barplot(x=top_varieties.values, y=top_varieties.index, palette='coolwarm')
plt.xlim(80,100)
plt.xlabel('Punteggio Medio')
plt.title('Top 10 Varietà con Miglior Punteggio Medio')
plt.show()

In [ ]:
fig = px.pie(
    popular_countries_df, 
    names='country', 
    title='Distribuzione Percentuale dei Paesi nel Dataset Iniziale',
)
fig.show()

In [ ]:
fig = px.pie(
    final_df, 
    names='country', 
    title='Distribuzione Percentuale dei Paesi nel Dataset Finale',
)
fig.show()

Dal confronto dei due grafici si evince che i paesi più frequenti nel dataframe finale (US, Francia e Italia) sono gli stessi del df iniziale.
Cambiano le percentuali sia di questi paesi che degli altri (probabilmente perchè abbiamo rimosso i vini che entravano nella fascia di prezzo 'expensive' degli Outlier), in particolare vediamo come il Cile si prende il quarto posto scavalcando Spagna e Portogallo, avvicinandosi alla percentuale dei vini Italiani 

In [ ]:
df_expensive['country'].value_counts()

In [ ]:
final_df['price'].describe()

Faccio una ulteriore analisi su prezzi e punteggi per vedere come questi sono variati rispetto al df iniziale

In [ ]:
final_df['price'].plot(kind='hist',bins=30, xlabel='Price', title='Distribuzione dei prezzi dei vini nel dataframe finale')

In [379]:
# La fascia di prezzo più comune è rimasta pressochè invariata ma abbiamo abbattuto notevolmente il prezzo massimo 

In [ ]:
final_df['points'].describe()

In [ ]:
final_df['points'].plot(kind='hist', bins=20)

In [380]:
# La miglior frequenza di punteggio è ora tra 87 e 93 punti, che rappresenta un notevole aumento rispetto al df iniziale

# CONCLUSIONI

Come detto in precedenza, per la costruzione del Marketplace viene considerato un target di persone che amano i vini di qualità ma che sono attanti al prezzo.  

Il dataframe finale, risultato concatenando tutti i subset ottenuti dall'analisi dei dati, contiene vini delle varietà e delle nazioni più popolari e più ricercate, vini delle vigne più apprezzate e più di nicchia e con le caratteristiche organolettiche più richieste.  
In generale esso è basato su vini che hanno ottenuto punteggi migliori e con prezzi più o meno contenuti, il rapporto qualità prezzo è stato preso in considerazione in tutte le analisi a rafforzare le richieste del nostro target.  
Dai grafici si vede come la fascia di prezzo più comune rimane pressochè invariata rispetto al df di partenza(10-40USD). E' stato abbattuto il prezzo massimo, che ora è di 160 USD rispetto a oltre 3000USD del df iniziale.
Si vede una certa tendenza di aumento di punteggio all'aumentare del prezzo ma ci sono tantissime alternative con ottimo rapporto qualità-prezzo.
Inoltre si puà notare come, rispetto al dataframe originario, la media punti sia aumentata e come la maggior parte dei prodotti (ovvero quelli compresi tra il primo e il terzo quartile) abbia punteggio tra 87 e 93 punti che rappresenta un notevole aumento della qualità dei vini presenti nel nostro dataframe.  

Tutto questo ci va ad assicurare che il nostro marketplace contiene prodotti delle varietà, dei paesi e delle vigne più richieste. Contiene prodotti di alta qualità per ogni fascia di prezzo rilevante per il nostro target, includendo i prodotti a più alta rotazione e più conosciuti dai potenziali clienti, oltre che qualche chicca per veri intenditori.